# Simple Example with Steam and Heat Exchangers

This notebook demonstrates how to assemble, initialize, and run a very simple flowsheet using steam and heat exchangers. This breifly demonstrates the basics of setting up a flowsheet.  See the documentation for a more complete description of the framework.

## Imports
Standard Pyomo and IDAES core components are imported below along with Panda with will be used in this example to print a table of steam states at different locations in the flowsheet.

In [1]:
import pyomo.environ as pe
from pyomo.network import Arc
from idaes.core import FlowsheetBlock, StateBlockBase
from idaes.unit_models import Heater, HeatExchanger
from idaes.unit_models.heat_exchanger import delta_temperature_amtd_rule
from idaes.property_models import iapws95_ph
import pandas as pd

## Create a Flowsheet

The lines below create a Pyomo concrete model, flowsheet and steam property parameter block instance. The steam property parameter block contains constants that can be shared among all similar state blocks in a flowsheet.

In [2]:
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(default={"dynamic": False})
model.fs.properties = iapws95_ph.Iapws95ParameterBlock()

## Create a Heater and HeatExchanger Model

Steam properties are used, notice in the heat exchanger model different properties can be used for each side. The default log-mean temperature difference was also swapped out for the arethmetic mean better behaved equations for this example.

In [3]:
model.fs.heater = Heater(default={"property_package": model.fs.properties})
model.fs.heat_exchanger = HeatExchanger(default={
        "delta_temperature_rule":delta_temperature_amtd_rule,
        "side_1":{"property_package": model.fs.properties},
        "side_2":{"property_package": model.fs.properties}})

## Fix Inlets and Initialize

In [4]:
# Fix heater inlets and heat duty then initialize unit
model.fs.heater.inlet[:].enth_mol.fix(4000)
model.fs.heater.inlet[:].flow_mol.fix(100)
model.fs.heater.inlet[:].pressure.fix(101325)
model.fs.heater.heat_duty[0].fix(100*20000)
model.fs.heater.initialize()

In [5]:
# Fix both heat exchanger intlets and initialize
model.fs.heat_exchanger.inlet_1[:].flow_mol.fix(100)
model.fs.heat_exchanger.inlet_1[:].pressure.fix(101325)
model.fs.heat_exchanger.inlet_1[:].enth_mol.fix(4000)
model.fs.heat_exchanger.inlet_2[:].flow_mol.fix(100)
model.fs.heat_exchanger.inlet_2[:].pressure.fix(101325)
model.fs.heat_exchanger.inlet_2[:].enth_mol.fix(3000)

model.fs.heat_exchanger.initialize()

## Solve the Disconnected Flowsheet

In [6]:
solver = pe.SolverFactory('ipopt')

In [7]:
solver.solve(model, tee=True)

Ipopt 3.12.11: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       21
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:       11
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       11
Total number of in

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 11, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.11\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.09377336502075195}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Write a Function to Show States at Ports

Write a function that will give a nice tabulated overview of the conditions in the process.

In [8]:
def state_table(m):
    head = ["State Block", 
        "Flow (mol/s)",
        "Temperature (K)",
        "Pressure (Pa)",
        "Enthalpy (J/mol)",
        "Vapor Fraction"]
    st = pd.DataFrame(columns=head)
    j = 0
    for c in model.component_objects():
        if isinstance(c, StateBlockBase):
            for i in c:
                row = [c[i].name, 
                       pe.value(c[i].flow_mol), 
                       pe.value(c[i].temperature), 
                       pe.value(c[i].pressure), 
                       pe.value(c[i].enth_mol),
                       pe.value(c[i].vapor_frac)]
                st.loc[j] = row
                j += 1
    return st

## Show States

Have a look at the results for the disconneted sheet.

In [15]:
state_table(model)

,State Block,Flow (mol/s),Temperature (K),Pressure (Pa),Enthalpy (J/mol),Vapor Fraction
0,fs.heater.control_volume.properties_in[0.0],100,326.166708,101325,4000,0.000000
1,fs.heater.control_volume.properties_out[0.0],100,373.124296,101325,24000,0.404678
2,fs.heat_exchanger.side_1.properties_in[0.0],100,373.124296,101325,24000,0.404678
3,fs.heat_exchanger.side_1.properties_out[0.0],100,334.275568,101325,4611.18,0.000000
4,fs.heat_exchanger.side_2.properties_in[0.0],600,312.888963,101325,3000,0.000000
5,fs.heat_exchanger.side_2.properties_out[0.0],600,355.733262,101325,6231.47,0.000000


## Conncect the Flowsheet

Now a flowsheet will be connected where steam will be generated in the heater model and used to heat water in the heat exchanger model.

In [10]:
model.fs.heat_exchanger.inlet_1[:].flow_mol.unfix()
model.fs.heat_exchanger.inlet_1[:].pressure.unfix()
model.fs.heat_exchanger.inlet_1[:].enth_mol.unfix()

model.fs.heat_exchanger.inlet_2[:].flow_mol.fix(600)

[None]

In [11]:
model.fs.stream_1 = Arc(source=model.fs.heater.outlet[0], 
                        destination=model.fs.heat_exchanger.inlet_1[0])

In [12]:
pe.TransformationFactory("network.expand_arcs").apply_to(model)

In [13]:
solver.solve(model, tee=True)

Ipopt 3.12.11: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       33
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       13

Total number of variables............................:       14
                     variables with only lower bounds:        8
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       14
Total number of in

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 14, 'Number of variables': 14, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.11\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.10614275932312012}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [14]:
state_table(model)

,State Block,Flow (mol/s),Temperature (K),Pressure (Pa),Enthalpy (J/mol),Vapor Fraction
0,fs.heater.control_volume.properties_in[0.0],100,326.166708,101325,4000,0.000000
1,fs.heater.control_volume.properties_out[0.0],100,373.124296,101325,24000,0.404678
2,fs.heat_exchanger.side_1.properties_in[0.0],100,373.124296,101325,24000,0.404678
3,fs.heat_exchanger.side_1.properties_out[0.0],100,334.275568,101325,4611.18,0.000000
4,fs.heat_exchanger.side_2.properties_in[0.0],600,312.888963,101325,3000,0.000000
5,fs.heat_exchanger.side_2.properties_out[0.0],600,355.733262,101325,6231.47,0.000000
